# 🚢 Trading RL Agent - Complete Containerization for K8s

## Objective
Demonstrate complete containerization of the trading RL pipeline:
- ✅ Containerized message broker stack (NATS + Redis)
- ✅ Docker builds for development and production
- ✅ Kubernetes manifest generation
- ✅ End-to-end pipeline validation

**Result**: Everything runs in containers, ready for K8s deployment.

In [ ]:
# Setup and imports
import json
from pathlib import Path
import subprocess
import sys
import time

PROJECT_ROOT = Path("/workspaces/trading-rl-agent")
sys.path.insert(0, str(PROJECT_ROOT / "src"))

print("🐳 Trading RL Agent - Complete Containerization Demo")
print("=" * 55)

## 1. Deploy Broker Infrastructure (Containerized)

Deploy NATS + Redis message broker stack using Docker containers.

In [ ]:
def deploy_broker_stack():
    """Deploy containerized NATS + Redis broker stack"""
    print("🚀 Deploying broker infrastructure...")

    # Start the broker stack
    result = subprocess.run(
        ["docker-compose", "-f", "docker-compose.broker.yml", "up", "-d"],
        capture_output=True,
        text=True,
        cwd=PROJECT_ROOT,
    )

    if result.returncode == 0:
        print("✅ Broker stack deployed successfully")
        print("  • NATS: localhost:4222 (client), localhost:8222 (monitoring)")
        print("  • Redis: localhost:6379")

        # Wait for services
        print("⏳ Waiting for services to be ready...")
        time.sleep(10)
        return True
    else:
        print(f"❌ Failed to deploy brokers: {result.stderr}")
        return False


# Deploy brokers
broker_deployed = deploy_broker_stack()

## 2. Build and Test Docker Images

Build Docker images for the trading pipeline and test containerized execution.

In [ ]:
def build_and_test_containers():
    """Build Docker images and test containerized pipeline"""
    print("🐳 Building Docker images...")

    # Build development image
    result = subprocess.run(
        ["docker", "build", "-f", "Dockerfile.dev", "-t", "trading-rl-dev", "."],
        capture_output=True,
        text=True,
        cwd=PROJECT_ROOT,
    )

    if result.returncode != 0:
        print(f"❌ Failed to build dev image: {result.stderr}")
        return False

    print("✅ Development image built successfully")

    # Test containerized pipeline
    print("🚀 Testing containerized pipeline...")
    test_result = subprocess.run(
        [
            "docker",
            "run",
            "--rm",
            "--network",
            "host",
            "-v",
            f"{PROJECT_ROOT}/data:/app/data",
            "trading-rl-dev",
            "python",
            "-c",
            """
import sys
sys.path.append('/app/src')
from data.synthetic import fetch_synthetic_data
from data.preprocessing import prepare_data
from data.features import generate_features
print('🔄 Running containerized pipeline...')
data = fetch_synthetic_data(n_samples=500, volatility=0.02)
processed = prepare_data(data)
features = generate_features(processed)
print(f'✅ Success: {len(features)} samples, {len(features.columns)} features')
""",
        ],
        capture_output=True,
        text=True,
    )

    if test_result.returncode == 0:
        print("✅ Containerized pipeline test successful")
        print(f"Output: {test_result.stdout.strip()}")
        return True
    else:
        print(f"❌ Pipeline test failed: {test_result.stderr}")
        return False


# Build and test
containers_ready = build_and_test_containers()

## 3. Generate Kubernetes Manifests

Create production-ready K8s deployment manifests.

In [ ]:
def generate_k8s_manifests():
    """Generate production-ready Kubernetes manifests"""
    print("☸️ Generating Kubernetes manifests...")

    manifests = {
        "namespace.yaml": """
apiVersion: v1
kind: Namespace
metadata:
  name: trading-rl
  labels:
    name: trading-rl
""",
        "nats-service.yaml": """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nats
  namespace: trading-rl
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nats
  template:
    metadata:
      labels:
        app: nats
    spec:
      containers:
      - name: nats
        image: nats:2.10-alpine
        ports:
        - containerPort: 4222
        - containerPort: 8222
        args: ["--http_port", "8222", "--jetstream"]
        resources:
          requests:
            memory: "256Mi"
            cpu: "250m"
          limits:
            memory: "512Mi"
            cpu: "500m"
---
apiVersion: v1
kind: Service
metadata:
  name: nats-service
  namespace: trading-rl
spec:
  ports:
  - port: 4222
    name: client
  - port: 8222
    name: monitoring
  selector:
    app: nats
""",
        "trading-job.yaml": """
apiVersion: batch/v1
kind: Job
metadata:
  name: trading-rl-training
  namespace: trading-rl
spec:
  template:
    spec:
      restartPolicy: Never
      containers:
      - name: trading-trainer
        image: trading-rl-prod:latest
        env:
        - name: NATS_URL
          value: "nats://nats-service:4222"
        resources:
          requests:
            memory: "2Gi"
            cpu: "1000m"
          limits:
            memory: "4Gi"
            cpu: "2000m"
""",
    }

    # Write manifests
    for filename, content in manifests.items():
        filepath = PROJECT_ROOT / f"k8s-{filename}"
        with open(filepath, "w") as f:
            f.write(content.strip())
        print(f"✅ Generated k8s-{filename}")

    print("📋 Kubernetes manifests ready for deployment!")
    return True


# Generate manifests
k8s_ready = generate_k8s_manifests()

## 4. Final Validation & Deployment Guide

Validate the complete setup and provide deployment instructions.

In [ ]:
def final_validation():
    """Final validation and deployment guide"""
    print("🔍 Final Infrastructure Validation")
    print("=" * 40)

    components = {
        "Broker Containers": broker_deployed,
        "Pipeline Containerization": containers_ready,
        "K8s Manifests": k8s_ready,
    }

    all_ready = True
    for component, status in components.items():
        icon = "✅" if status else "❌"
        print(f"{icon} {component}: {'READY' if status else 'FAILED'}")
        if not status:
            all_ready = False

    print("\n💰 Cost-Effective Solution:")
    print("🏆 NATS + Redis (Containerized)")
    print("   • Development: $0 (local containers)")
    print("   • Production: ~$50-200/month (managed K8s)")
    print("   • Lightweight, fast, K8s-native")

    if all_ready:
        print("\n🚀 K8s Deployment Commands:")
        print("kubectl apply -f k8s-namespace.yaml")
        print("kubectl apply -f k8s-nats-service.yaml")
        print("kubectl apply -f k8s-trading-job.yaml")
        print("\n🎉 SYSTEM IS K8S-READY!")
    else:
        print("\n❌ Fix failed components before K8s deployment")

    return all_ready


# Final validation
system_ready = final_validation()
print(f"\n{'🎯 READY FOR PRODUCTION' if system_ready else '⚠️ NEEDS FIXES'}")

## Summary

This notebook demonstrates a complete containerized setup for the Trading RL Agent:

1. **Containerized Brokers**: NATS + Redis running in Docker containers
2. **Pipeline Containerization**: Complete trading pipeline tested in containers
3. **K8s Readiness**: Production manifests generated for deployment
4. **Cost Effective**: ~$50-200/month for production vs $500+ for managed services

**Next Steps**: Push images to registry and deploy to Kubernetes cluster.